In [ ]:
import glob
import cupy as cp
import numpy as np
from datetime import datetime
from hist._cmd_line_util import open_input_image
from hist.exact import histeq_exact, __calc_info
from utils import clear_line


exact_histeq_methods = ['lm', 'va']

for method in exact_histeq_methods:
    # 2D and 3D Images
    images = glob.glob('../c4l-image-dataset/*/*')
    n_imgs = len(images)
    for i, filename in enumerate(images):
        string = f'Progress: {i}/{n_imgs}\tCurrent image: {filename[21:]}'
        print(string, end='')

        # Setup for function calls
        timing_data = []
        img_cpu = open_input_image(filename)
        img_gpu = cp.array(img_cpu)


        # GPU timing
        data = %timeit -q -o __calc_info(img_gpu, method)
        timing_data.append(data)

        data = %timeit -q -o histeq_exact(img_gpu, method=method)
        timing_data.append(data)


        # CPU timing
        data = %timeit -q -o __calc_info(img_cpu, method)
        timing_data.append(data)

        data = %timeit -q -o histeq_exact(img_cpu, method=method)
        timing_data.append(data)

        # Save data in file; appends to the end of the file
        with open(method + '_temp.txt', 'a+') as file:
            file.write('\n\n' + filename + '\n')
            for data in timing_data:
                file.write(str(data) + '\n')

        clear_line(len(string)) # Clears the current output

print('Finished!')
        

In [ ]:
import os
import os.path
from utils import prefix_to_micro


def convert_temp_to_data(method):
    if os.path.exists(method + '_temp.txt'):
        with open(method + '_temp.txt', 'r') as temp:
            with open(method + '_timing.csv', 'w+') as file:
                file.write('Image,__calc_info_gpu Mean,__calc_info_gpu Std. Dev.,histeq_exact_gpu Mean, histeq_exact_gpu Std. Dev.,'
                           '__calc_info_cpu Mean,__calc_info_cpu Std. Dev.,histeq_exact_cpu Mean,histeq_exact_cpu Std. Dev.\n')
                data = temp.read().strip().split('\n\n')
                for img in data:
                    img = img.strip().split('\n')
                    data_list = [img[0].strip('..').strip('/')]
                    # Get time values and convert them to microseconds
                    for i in range(1, 5):
                        vals = img[i].split(' ')
                        data_list.append(float(vals[0])*prefix_to_micro(vals[1]))
                        data_list.append(float(vals[3])*prefix_to_micro(vals[4]))
                    file.write(','.join([str(_) for _ in data_list]) + '\n')
            
    #os.remove(method + '_temp.txt')

In [ ]:
exact_histeq_methods = ['lm', 'va']

for method in exact_histeq_methods:
    convert_temp_to_data(method)